# 目标检测 Object Detection

## 1. 目标定位 Object Localization

目标检测是这几年随着深度学习兴起以来，计算机视觉领域中进展神速的一个问题。要进行目标检测，目标定位是第一步。

目标定位就是在一个图像中，找到目标，用一个红框将其标出，目标定位通常和分类器配合使用，只涉及一个目标。而目标检测则在此基础上，进行多个目标的标记。

![What are localization and detection.png](img/What are localization and detection.png)

带目标定位的神经网络分类器，就是在卷积神经网络分类器的基础上，在最后的Softmax层，加入红框的坐标信息。这里记做 $b_x, b_y, b_h, b_w$，分别表示红框中心的横坐标，红框中心的纵坐标，红框的高度和红框的宽度。

![Classification with localization.png](img/Classification with localization.png)

最终输出的预测值 $\hat{y}$，包括三个部分：$p_c$ 图像中是否包含目标，$b_x, b_y, b_h, b_w$ 红框坐标信息，以及 $c_1, c_2, c_3$ 分类器分类信息。当图像中不包含目标时，红框坐标信息和分类信息实际上是我们不关心的，所以损失函数也会相应地调整，在 $y=0$ 时，只计算 $p_c$ 的误差。下面的损失函数以误差平方为例，考虑到是否包含目标以及分类信息都属于分类问题，这两部分的损失也可以用logloss来计算。

![Defining the target label y.png](img/Defining the target label y.png)

## 2. 特征点检测 Landmark detection

类似上面带目标定位的神经网络分类器，也可以让神经网络最终的输出层直接输出**特征点 Landmark**的坐标。但特征点的数量需要预先定义。并且每个特征点在输入向量中的位置（比如左眼内侧眼角，下巴，膝盖），需要跨样本保持一致。

![Landmark detection.png](img/Landmark detection.png)